In [2]:
# default_exp facade

In [10]:
!pip install nbdev
!pip install sentencepiece
!pip install dit

You should consider upgrading via the '/Users/danielquiroga/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/danielquiroga/anaconda3/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 397 kB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 170 kB 2.9 MB/s eta 0:00:01


     |████████████████████████████████| 106 kB 4.9 MB/s eta 0:00:01
You should consider upgrading via the '/Users/danielquiroga/anaconda3/bin/python -m pip install --upgrade pip' command.


In [14]:
import random
from nbdev.showdoc import *
#from ds4se.mgmnt.prep.bpe import *
#from ds4se.exp.info import *
#from ds4se.desc.stats import *
import pandas as pd
import sentencepiece as sp
from pathlib import Path
from collections import Counter
from ds4se.mining.unsupervised.traceability.eval import *

In [15]:
import pandas as pd

In [16]:
#export
def TraceLinkValue(source, target, technique):
    #param source
    #param target
    #param technique
    #return 
    
    #TODO make the string saved and then return the path to that string
    sourcePath = "path/to/something"#this is where we save the passed in source
    targetPath = "./something"
    
    if (technique == "VSM"):
        pass
    if (technique == "LDA"):
        pass
    if (technique == "orthogonal"):
        pass
    if (technique == "LSA"):
        pass
    if (technique == "JS"):
        pass
    if (technique == "word2vec"):
        parameter = {
            "vectorizationType": VectorizationType.word2vec,
            "linkType": LinkType.req2tc,
            "system": 'libest',
            "path_to_trained_model": 'test_data/models/word2vec_libest.model',
            "source_path": sourcePath,
            "target_path": targetPath,
            "system_path": '/tf/main/benchmarking/traceability/testbeds/nltk/[libest-pre-all].csv',
            "saving_path": 'test_data/',
            "names": ['Source','Target','Linked?']
        }
        pass
    if (technique == "doc2vec"):
        parameter = {
            "vectorizationType": VectorizationType.doc2vec,
            "linkType": LinkType.req2tc,
            "system": 'libest',
            "path_to_trained_model": 'test_data/models/doc2vec_libest.model',
            "source_path": source,
            "target_path": target,
            "system_path": '/tf/main/benchmarking/traceability/testbeds/nltk/[libest-pre-all].csv',
            "saving_path": 'test_data/',
            "names": ['Source','Target','Linked?']
        }
        pass
    value = random.randint(0,1)/100
    
    return value

#export
def NumDoc(source, target):
    source_doc = source.shape[0]
    target_doc = target.shape[0]
    difference = source_doc - target_doc
    return [source_doc, target_doc, difference, -difference]
   
#export
def VocabSize(source, target):
    source_list = preprocess(source)
    target_list = preprocess(target)
    source_size = len(source_list[0])
    target_size = len(target_list[0])
    difference = source_size - target_size
    return [source_size, target_size, difference, -difference]  

#export
def AverageToken(source, target):
    source_doc = source.shape[0]
    target_doc = target.shape[0]
    
    source_list = preprocess(source)
    target_list = preprocess(target)
    
    source_total_token = sum(source_list[0].values())
    target_total_token = sum(target_list[0].values())

    source_token = source_total_token/source_doc
    target_token = target_total_token/target_doc
    difference = source_token - target_token
    return [source_token, target_token, difference, -difference]

#export
def Vocab(artifacts_df):
    #we can add a parameter for user to specify the number of most frequent token to return
    cnts = preprocess(artifacts_df)
    vocab_list = cnts[0].most_common(3)
    total = sum(cnts[0].values())
    vocab_dict = dict()
    vocab_dict[vocab_list[0][0]] = [vocab_list[0][1], vocab_list[0][1]/total]
    vocab_dict[vocab_list[1][0]] = [vocab_list[1][1], vocab_list[1][1]/total]
    vocab_dict[vocab_list[2][0]] = [vocab_list[2][1], vocab_list[2][1]/total]

    return vocab_dict
    
    
#export
def VocabShared(source, target):
    df = pd.concat([source, target])
    return Vocab(df) 
    
#export
def SharedVocabSize(source, target):
    df = pd.concat([source, target])
    df_counts = preprocess(df)
    shared_size = len(df_counts[0])
    return shared_size

#export
def MutualInformation(source, target):
    mutual_information = random.randint(100,200)
    return mutual_information
    
#export
def CrossEntropy(source, target):
    cross_entropy = random.randint(100,200)
    cross_entropy = get_system_entropy_from_df(source, "col1",)
    return cross_entropy
    
#export
def KLDivergence(source, target):
    divergence = random.randint(100,200)
    return divergence

#export
def get_docs(df, spm):
    docs = []
    for fn in df["col1"]:
        docs += spm.EncodeAsPieces(fn)
    return docs

#export
def get_counters(docs):
    doc_cnts = []
    cnt = Counter()
    for tok in docs:
        cnt[tok] += 1 
        doc_cnts.append(cnt)
    return doc_cnts

#export
def preprocess(artifacts_df):
    spm = sp.SentencePieceProcessor()
    output = Path('test_data\models')
    system_name = "test"
    spm.Load(f"{output / system_name}.model")
    docs = get_docs(artifacts_df,spm)
    cnts = get_counters(docs)
    return cnts

In [272]:
# df = pd.DataFrame({
#     "col1": ["one, two two am one,"]})
# df2 = pd.DataFrame({
#     "col1": ["this is a requirement yes or no two two two two one,"]})
# frames = [df, df2]
# df3 = pd.concat(frames)
# VocabShared(df,df2)

In [135]:
# import pandas as pd
# import sentencepiece as sp
# from pathlib import Path
# from collections import Counter


# df = pd.DataFrame({
#     "col1": ["this is a sentence", "this is another sentence"],
#     "col2": ["one more", "this is the last sentence"],
# })


In [136]:
# def get_docs(df, spm):
#     docs = []
#     for fn in df["col1"]:
#         docs += spm.EncodeAsPieces(fn)
#     return docs
# d = {'col1': ["one requirement, i want this and that and this", "this is is is is a new requirement"], 'col2': [3, 4]}
# df = pd.DataFrame(data=d)


# docs

In [148]:
def get_counters(docs):
    doc_cnts = []
    cnt = Counter()
    for tok in docs:
        cnt[tok] += 1 
        doc_cnts.append(cnt)
    return doc_cnts

req_cnts = get_counters(docs)
list = req_cnts[0].most_common(2)
sum(req_cnts[0].values())


34

In [93]:
# nbdev_build_docs
# notebook2script()

In [ ]:
#FacadeTest
#Test case for proto TLV

#Prototype should print a value between 0 or 1. Input values aren't used
testTLV = TraceLinkValue(0,1,1)
assert((testTLV == 0) or (testTLV == 1))

In [ ]:
#FacadeTest
#Test Case for Proto NumDoc

#Prototype should print an array based on the randomly generated numbers. Input values aren't used.
testND = NumDoc(100,10000000)
assert(isinstance(testND, list))

In [ ]:
#FacadeTest
#Test case for Vocab size

#Prototype should return an array based on random number values. Inputs aren't used.
testVS = VocabSize(100,100000)
assert(isinstance(testVS, list))

In [ ]:
#FacadeTest
#Test Case for AverageToken

#Prototype should return an array/list. Inputs aren't used
testAT = AverageToken(100,100)
assert(isinstance(testAT, list))

In [ ]:
#FacadeTest
#Test Case for Vocab

#Prototype Vocab should return dictionary type. 
#Will expand to test dictionary values once presets can be generated and tested
testVocab = Vocab("file")

assert(isinstance(testVocab,dict))

In [ ]:
#FacadeTest
#Test VocabShared

#Prototype should show a dictionary
#Test will be expanded to verify frequency verification in two samples
testVocabS = VocabShared("source","target")
assert(isinstance(testVocabS,dict))


In [ ]:
#FacadeTest
#Test Shared vocab size

#Prototype should return an i
testSVS = SharedVocabSize("source","target")
assert(testSVS in range(100,200))

In [ ]:
#FacadeTest
#TEst Mutual information

#Proto mutual information should get an int
testMutualInfo = MutualInformation("source","target")
assert(testMutualInfo in range(100,200))

In [ ]:
#FacadeTest
#Test Cross Entropy

#Proto cross Entropy should return an int
testCrossEntropy = CrossEntropy("Sauce","Target")
assert(testCrossEntropy in range(100,200))

#Eventually cross entropy will properly implement Entropy calculation and the test will compare a known/predicted entropy
# to a calculated value


In [ ]:
#FacadeTest
#Test KLDivergence

#Proto KLDivegence will return an random int
testKLDiver = KLDivergence("source","target")
assert(testKLDiver in range(100,200))